In [1]:
import pandas as pd
from glob import glob
from os import listdir, chdir
from re import compile as rcompile
from path import Path
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, Nadam

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def read_transcript(file_name):
    file_names = glob(f'{file_name}*')
    ret = ''
    for fname in file_names:
        with open(fname) as fd:
            ret += fd.read()+'\n\n'
    return ret

reference_transcripts_dir = '../data/transcripts'
google_transcripts_dir = '../data/google_transcripts'

reference_transcripts_files = listdir(reference_transcripts_dir)
google_transcripts_files = listdir(google_transcripts_dir)
reference_transcripts_files.sort()
google_transcripts_files.sort()

In [3]:
print(reference_transcripts_files)
print(google_transcripts_files)

['.ipynb_checkpoints', '3101.txt', '3102.txt', '3103.txt', '3104.txt', '3105.txt', '3106.txt', '3107.txt', '3108.txt', '3109.txt', '3110.txt', '3111.txt', '3112.txt', '3113.txt', '3114.txt', '3115.txt', '3116.txt', '3117.txt', '3118.txt', '3119.txt', '3120.txt', '3121.txt', '3122.txt', '3123.txt', '3124.txt', '3125.txt', '3126.txt', '3127.txt', '3128.txt', '3129.txt', '3130.txt', '3131.txt', '3132.txt', '3201.txt', '3202.txt', '3203.txt', '3204.txt', '3205.txt', '3206.txt', '3207.txt', '3208.txt', '3209.txt', '3210.txt', '3211.txt', '3212.txt', '3213 (missing page 2).txt', '3214.txt', '3215 (missing page 2).txt', '3216.txt', '3217.txt', '3218.txt', '3219.txt', '3220.txt', '3221.txt', '3222.txt', '3223.txt', '3224.txt', '3225.txt', '3226.txt', '3227.txt', '3228.txt', '3229.txt', '3230.txt', '3231.txt', '3232.txt', '3234.txt', '3235.txt', '3236.txt', '3237.txt', '3238.txt', '3239.txt', '3240.txt', '3241.txt', '3243.txt', '3244.txt', '3245.txt', '3246.txt', '3247.txt', '3248.txt', '5101.t

In [4]:
#Some of the reference transcripts have missing information and those files have
#filename with '(missing page <number>)' substrings - filter these files out for
#now.
rex = rcompile('[0-9]+.txt')
reference_transcripts_file_roots = [x.split('.')[0] for x in reference_transcripts_files if rex.match(x) is not None]
print(reference_transcripts_file_roots)

['3101', '3102', '3103', '3104', '3105', '3106', '3107', '3108', '3109', '3110', '3111', '3112', '3113', '3114', '3115', '3116', '3117', '3118', '3119', '3120', '3121', '3122', '3123', '3124', '3125', '3126', '3127', '3128', '3129', '3130', '3131', '3132', '3201', '3202', '3203', '3204', '3205', '3206', '3207', '3208', '3209', '3210', '3211', '3212', '3214', '3216', '3217', '3218', '3219', '3220', '3221', '3222', '3223', '3224', '3225', '3226', '3227', '3228', '3229', '3230', '3231', '3232', '3234', '3235', '3236', '3237', '3238', '3239', '3240', '3241', '3243', '3244', '3245', '3246', '3247', '3248', '5101', '5102', '5103', '5104', '5105', '5106', '5107', '5108', '5109', '5110', '5111', '5112', '5113', '5114', '5115', '5116', '5117', '5118', '5119', '5120', '5121', '5122', '5123', '5124', '5125', '5126', '5129', '5130', '5131', '5132', '5202', '5203', '5204', '5205', '5206', '5207', '5208', '5209', '5210', '5213', '5214', '5215', '5216', '5217', '5218', '5219', '5220', '5221', '5222',

In [5]:

with Path(reference_transcripts_dir):
    reference_transcripts = [read_transcript(f'{x}.txt') for x in reference_transcripts_file_roots]

print(len(reference_transcripts),'\n', reference_transcripts[0],'\n', reference_transcripts[1])

with Path(google_transcripts_dir):
    google_transcripts = [read_transcript(x) for x in reference_transcripts_file_roots]
    
print(len(google_transcripts),'\n', google_transcripts[0],'\n', google_transcripts[1])


165 
 Once apon a time there was a girl named Mary. On a warm 
sunny day Mary was walking through the woods near hear nouse 
to look for some critters to take pictures of. She loved animals 
and nature all her life even though she was only nine years old 
She thinks that she is going to die soon. She does go to school 
but she isn't that smart. For example Mary recycled a pie 
even though the pie was not even bitten. Another thing she did 
was leave her muffin in the woods while she was traveling with  
her family to Andrea's house which is one of Mary's friends at 
school and Mary's muffin was eaten by a bear that sniffed it :q!
:
from far away. Yes. Mary could be a little weird but that is 
just how she is. Back to the real story now. Mary found a 
fox, took a picture she also found a rabbit took a picture 
she found a squirrel and took a picture before it ran away. 
But sooner or later she saw two eyes peeking through a bush 
on the side of the path she was walking on. Then the 
cre

In [6]:
# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english',
                        ngram_range=(1,2),
                        min_df=1
                       )
# Create a vocabulary and get word counts per document
ref_dtm = tfidf.fit_transform(reference_transcripts)

features = tfidf.get_feature_names()
#display(len(features), features[:50])

# Get feature names to use as dataframe column headers
ref_dtm = pd.DataFrame(ref_dtm.todense(), columns=features)
print(ref_dtm.shape)
display(ref_dtm.head())


(165, 21876)


,00,00 meet,00 morning,00 said,00 star,00 telling,00 time,000,000 000,000 5203,...,zoe told,zoe wasn,zoe worms,zoomed,zoomed planted,zoomed shop,zrm,zrm jedi,zzzzzz,zzzzzz wake
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.051429,0.051429
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [7]:
ishape = ref_dtm.shape[1]
# Create Model 
input_img = Input(shape=(ishape, ))

x = Dense(1024)(input_img)

x = Dense(256)(x)

x = Dense(128)(x)

encoded = Dense(64)(x)

x = Dense(128)(encoded)

x = Dense(256)(x)

x = Dense(1024, activation='sigmoid')(x)
decoded = Dense(ishape, activation='sigmoid')(x)


rmodel = Model(input_img, decoded)
rmodel.compile(loss='mse', optimizer=Adam(learning_rate=0.01))

rmodel.fit(ref_dtm, ref_dtm, batch_size=512, epochs=4)

Epoch 1/4
1/1 [==============================] - 1s 750ms/step - loss: 0.2509
Epoch 2/4
1/1 [==============================] - 0s 223ms/step - loss: 0.0013
Epoch 3/4
1/1 [==============================] - 0s 230ms/step - loss: 5.6224e-05
Epoch 4/4
1/1 [==============================] - 0s 225ms/step - loss: 4.5460e-05


In [8]:
encoder = Model(input_img, encoded)

encoded_ref_dtm = encoder.predict(ref_dtm)
print(encoded_ref_dtm.shape)
display(encoded_ref_dtm)

(165, 64)


array([[-0.20209184,  0.04470556, -0.02643825, ..., -0.38869217,
        -0.25289783,  0.3736603 ],
       [-0.2060911 ,  0.03171282,  0.00361387, ..., -0.38009903,
        -0.22033359,  0.40168595],
       [-0.21420166,  0.0238283 , -0.01913108, ..., -0.3860425 ,
        -0.23833396,  0.39744115],
       ...,
       [-0.19514069,  0.04164192,  0.00533963, ..., -0.36928338,
        -0.24332632,  0.35344464],
       [-0.17044368,  0.04908935, -0.01003704, ..., -0.39402628,
        -0.23826472,  0.38615996],
       [-0.18238866,  0.05546333, -0.01705509, ..., -0.3944215 ,
        -0.22832987,  0.3948711 ]], dtype=float32)

In [9]:
def get_vector(doc):
    vec = tfidf.transform([doc]).todense()
    return encoder.predict(vec)

In [10]:
print(get_vector(google_transcripts[0]))

[[-0.19826442  0.04513413 -0.02046486  0.09756547 -0.2748718   0.03936278
   0.14605512 -0.3047451   0.01073894 -0.28902507  0.02956281 -0.2117703
   0.1251549  -0.17435195  0.2520424  -0.11814192  0.18164337 -0.1066189
   0.2782002   0.20693465  0.07913922 -0.0063188   0.06193272  0.11759567
   0.35405606  0.38036788 -0.2576433   0.3372022   0.0667557   0.08950295
   0.22679016 -0.01132899 -0.08049136 -0.05315205 -0.14614578  0.07444175
   0.10548063  0.16759844 -0.20775996  0.30337945  0.15599829 -0.28564122
  -0.25025833 -0.0936008   0.10519032 -0.04059128  0.13553895 -0.24569243
  -0.14977553  0.01828392 -0.14496695 -0.00780673 -0.08907203  0.03033754
  -0.06769946 -0.01583442  0.00582847  0.27126306 -0.06428409 -0.3049555
   0.23848267 -0.39359912 -0.25245962  0.37320215]]


In [17]:
def get_encoded_dtm(transcripts):
    dtm = tfidf.transform(transcripts).todense()
    print(dtm.shape)
    encoded_dtm = encoder.predict(dtm)
    print(encoded_dtm.shape)
    display(encoded_dtm)
    return encoded_dtm

encoded_google_dtm = get_encoded_dtm(google_transcripts)
print(encoded_google_dtm.shape)
display(encoded_google_dtm)

(165, 21876)
(165, 64)


array([[-0.19826454,  0.04513416, -0.02046483, ..., -0.393599  ,
        -0.2524596 ,  0.37320215],
       [-0.21015924,  0.02582057,  0.00458476, ..., -0.3815539 ,
        -0.23613209,  0.40191668],
       [-0.22374544,  0.02300805, -0.02269058, ..., -0.36834866,
        -0.21772985,  0.4085685 ],
       ...,
       [-0.1956769 ,  0.03437908,  0.00996523, ..., -0.36309996,
        -0.24307577,  0.3514322 ],
       [-0.17025381,  0.05206164, -0.00691394, ..., -0.38990152,
        -0.24444723,  0.3846683 ],
       [-0.18380612,  0.0261796 , -0.02427699, ..., -0.38227686,
        -0.2154612 ,  0.38472784]], dtype=float32)

(165, 64)


array([[-0.19826454,  0.04513416, -0.02046483, ..., -0.393599  ,
        -0.2524596 ,  0.37320215],
       [-0.21015924,  0.02582057,  0.00458476, ..., -0.3815539 ,
        -0.23613209,  0.40191668],
       [-0.22374544,  0.02300805, -0.02269058, ..., -0.36834866,
        -0.21772985,  0.4085685 ],
       ...,
       [-0.1956769 ,  0.03437908,  0.00996523, ..., -0.36309996,
        -0.24307577,  0.3514322 ],
       [-0.17025381,  0.05206164, -0.00691394, ..., -0.38990152,
        -0.24444723,  0.3846683 ],
       [-0.18380612,  0.0261796 , -0.02427699, ..., -0.38227686,
        -0.2154612 ,  0.38472784]], dtype=float32)

In [18]:
cosine_similarity([encoded_ref_dtm[0]], [encoded_google_dtm[0]])

array([[0.9996451]], dtype=float32)

In [20]:
def get_cossim(encoded_dtm):
    cossim = []
    for idx,ref in enumerate(encoded_ref_dtm):
        cossim.append(cosine_similarity([ref], [encoded_dtm[idx]])[0][0]) 
    return cossim

cossim = get_cossim(encoded_google_dtm)
print(cossim)

[0.9996451, 0.9987583, 0.997483, 0.99449646, 0.99939185, 0.9998019, 0.99860454, 0.9993582, 0.99924624, 0.998436, 0.99978673, 0.99825656, 0.9986698, 0.9992837, 0.99858224, 0.99908984, 0.9959377, 0.99801373, 0.998486, 0.999632, 0.99927014, 0.9997517, 0.9990992, 0.99786437, 0.99996185, 0.99952483, 0.99991333, 0.9974342, 0.999168, 0.9996357, 0.9996481, 0.9993719, 0.9982471, 0.99945366, 0.9997749, 0.99926823, 0.9995072, 0.99873954, 0.9994596, 0.99968684, 0.9994625, 0.9993816, 0.9995769, 0.99904704, 0.9998536, 0.9982956, 0.99783766, 0.99931526, 0.9995854, 0.999362, 0.9994318, 0.9988151, 0.9993373, 0.999783, 0.9995308, 0.99857354, 0.9995926, 0.9949379, 0.9991705, 0.9994416, 0.9996401, 0.9996173, 0.9988978, 0.99549675, 0.9938877, 0.9995114, 0.9995142, 0.99974906, 0.9969016, 0.9997885, 0.9984046, 0.9986464, 0.9993672, 0.9990442, 0.9994365, 0.99990535, 0.99896353, 0.9988198, 0.9995659, 0.9985357, 0.99840367, 0.99985087, 0.9996724, 0.9994431, 0.99833024, 0.99830306, 0.99973166, 0.9996222, 0.99798